# A* Mario Eduardo Sánchez Mejía

Vamos a crear el siguiente proyecto, que consta en filas de 10 y columnas de 10. solucionado por A*, El Ejercicio consta de lo siguiente. Hay un punto inicial ubicado en cualquier parte del tablero de 10x10, y un punto final ubicado en cualquier parte del 10x10 que no sea la posición de inicio. Y posterior a ellos hay ejercicios.

Primero vamos a hacer las importaciones necesarias. Y tener los links de las documentaicones a la mano [PyGames Documentation](https://www.pygame.org/docs/),
Usamos una herramienta que nos permite administrar colas, lo que significa es que la prioridad no va a ser de menor a mayor va a ser de mayor a menor. El nombre de esta herramienta es [PriorityQueue](https://docs.python.org/3/library/queue.html) y para mostrar la pestaña 

In [76]:
import pygame
from queue import PriorityQueue
import pygame.display

## Inicialización de Nuestro PyGames
Vamos a inicializar nuestro pygames, y definimos el alto y ancho, seteamos dichos valores y le colocamnos el título en este caso "Tarea A* Mario Eduardo Sánchez Mejía"

In [77]:
pygame.init()
ANCHO_VENTANA = 1000  
ALTO_VENTANA = 600
VENTANA = pygame.display.set_mode((ANCHO_VENTANA, ALTO_VENTANA))
pygame.display.set_caption("Tarea A* Mario Eduardo Sánchez Mejía 21120721")

## Declaración de Colores
* Color Blanco -> Fondo de nuestro juego.
* Color Negro -> Letras de Nuestos cualculos, color de los sumideros.
* Color Gris -> Color de la cuadricula.
* Color Rojo -> Tachado de la solución y el color en que se enmarca el camino a seguir.
* Azul Claro -> Es el fondo de las listas abiertas y cerradas.

In [78]:
BLANCO = (255, 255, 255)
NEGRO = (0, 0, 0)
GRIS = (200, 200, 200)
ROJO = (255, 0, 0)
AZUL_CLARO = (200, 220, 255)

## Declaración del tipo de Letra y Tamaño
* La primera es para los calculos necesarios de (F,G y H) y para los numeros de los recuadros.
* La segunda es para las listas cerradas y listas abiertas.
* La última es para los textos de "Inicio" y "Fin".

In [79]:
pygame.font.init()
FUENTE_PEQUENA = pygame.font.SysFont('Arial', 10)  
FUENTE_NORMAL = pygame.font.SysFont('Arial', 12)   
FUENTE_LISTA = pygame.font.SysFont('Arial', 14) 

In [80]:
class Nodo:
    def __init__(self, fila, col, ancho, total_filas, numero_secuencial):
        self.fila = fila
        self.col = col
        self.x = col * ancho
        self.y = fila * ancho
        self.ancho = ancho
        self.total_filas = total_filas
        self.numero_secuencial = numero_secuencial
        self.vecinos = []
        self.es_muro = False
        self.es_nodo_inicio = False
        self.es_nodo_fin = False
        self.visitado = False
        self.en_camino = False
        self.nodo_origen = None
        self.g = float("inf")
        self.h = 0
        self.f = float("inf")
    def get_pos(self):
        return self.fila, self.col
    def get_id(self):
        return str(self.numero_secuencial)
    def restablecer(self):
        self.es_muro = False
        self.es_nodo_inicio = False
        self.es_nodo_fin = False
        self.visitado = False
        self.en_camino = False
        self.nodo_origen = None
        self.g = float("inf")
        self.h = 0
        self.f = float("inf")
    def hacer_inicio(self):
        self.es_nodo_inicio = True
    def hacer_fin(self):
        self.es_nodo_fin = True
    def hacer_muro(self):
        self.es_muro = True
    def marcar_visitado(self):
        self.visitado = True
    def marcar_camino(self):
        self.en_camino = True
    def dibujar(self, ventana):
        if self.es_muro:
            pygame.draw.rect(ventana, NEGRO, (self.x, self.y, self.ancho, self.ancho))
        else:
            pygame.draw.rect(ventana, BLANCO, (self.x, self.y, self.ancho, self.ancho))
            if not self.es_nodo_inicio and not self.es_nodo_fin:
                id_texto = FUENTE_PEQUENA.render(self.get_id(), True, NEGRO)
                ventana.blit(id_texto, (self.x + 2, self.y + 2))
            if self.f != float("inf") and not self.es_nodo_inicio and not self.es_nodo_fin:
                g_texto = FUENTE_PEQUENA.render(f"g:{int(self.g)}", True, NEGRO)
                h_texto = FUENTE_PEQUENA.render(f"h:{int(self.h)}", True, NEGRO)
                f_texto = FUENTE_PEQUENA.render(f"f:{int(self.f)}", True, NEGRO)
                ventana.blit(g_texto, (self.x + 5, self.y + self.ancho//2 - 15))
                ventana.blit(h_texto, (self.x + 5, self.y + self.ancho//2))
                ventana.blit(f_texto, (self.x + 5, self.y + self.ancho//2 + 15))
            if self.visitado and not self.es_nodo_inicio and not self.es_nodo_fin:
                visitado_texto = FUENTE_NORMAL.render("*", True, NEGRO)
                ventana.blit(visitado_texto, (self.x + self.ancho - 12, self.y + 3))
            if self.es_nodo_inicio:
                inicio_texto = FUENTE_NORMAL.render("INICIO", True, NEGRO)
                ventana.blit(inicio_texto, (self.x + (self.ancho - inicio_texto.get_width()) // 2, 
                                            self.y + (self.ancho - inicio_texto.get_height()) // 2))
            
            if self.es_nodo_fin:
                fin_texto = FUENTE_NORMAL.render("FIN", True, NEGRO)
                ventana.blit(fin_texto, (self.x + (self.ancho - fin_texto.get_width()) // 2, 
                                        self.y + (self.ancho - fin_texto.get_height()) // 2))
            
            if self.en_camino:
                pygame.draw.rect(ventana, ROJO, (self.x, self.y, self.ancho, self.ancho), 3)
    def actualizar_vecinos(self, grid):
        self.vecinos = []
        if self.fila < self.total_filas - 1 and not grid[self.fila + 1][self.col].es_muro:
            self.vecinos.append((grid[self.fila + 1][self.col], 10, "abajo"))
        if self.fila > 0 and not grid[self.fila - 1][self.col].es_muro:
            self.vecinos.append((grid[self.fila - 1][self.col], 10, "arriba"))

        if self.col < self.total_filas - 1 and not grid[self.fila][self.col + 1].es_muro:
            self.vecinos.append((grid[self.fila][self.col + 1], 10, "derecha"))
        if self.col > 0 and not grid[self.fila][self.col - 1].es_muro:
            self.vecinos.append((grid[self.fila][self.col - 1], 10, "izquierda"))
        if (self.fila < self.total_filas - 1 and self.col < self.total_filas - 1 and 
            not grid[self.fila + 1][self.col + 1].es_muro and 
            not grid[self.fila + 1][self.col].es_muro and 
            not grid[self.fila][self.col + 1].es_muro):
            self.vecinos.append((grid[self.fila + 1][self.col + 1], 14, "diagonal-dr"))
        
        if (self.fila < self.total_filas - 1 and self.col > 0 and 
            not grid[self.fila + 1][self.col - 1].es_muro and 
            not grid[self.fila + 1][self.col].es_muro and 
            not grid[self.fila][self.col - 1].es_muro):
            self.vecinos.append((grid[self.fila + 1][self.col - 1], 14, "diagonal-di"))
        
        if (self.fila > 0 and self.col < self.total_filas - 1 and 
            not grid[self.fila - 1][self.col + 1].es_muro and 
            not grid[self.fila - 1][self.col].es_muro and 
            not grid[self.fila][self.col + 1].es_muro):
            self.vecinos.append((grid[self.fila - 1][self.col + 1], 14, "diagonal-ar"))
        
        if (self.fila > 0 and self.col > 0 and 
            not grid[self.fila - 1][self.col - 1].es_muro and 
            not grid[self.fila - 1][self.col].es_muro and 
            not grid[self.fila][self.col - 1].es_muro):
            self.vecinos.append((grid[self.fila - 1][self.col - 1], 14, "diagonal-ai"))
    def __lt__(self, other):
        if self.f == other.f:
            return self.g > other.g
        return self.f < other.f

In [81]:
def h(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return 10 * (abs(x1 - x2) + abs(y1 - y2))

In [82]:
def reconstruir_camino(came_from, actual, dibujar, lista_abierta, lista_cerrada):
    nodos_camino = set()
    
    while actual in came_from:
        actual.marcar_camino()
        nodos_camino.add(actual)
        actual = came_from[actual]
    
    actual.marcar_camino()
    nodos_camino.add(actual)
    
    dibujar(lista_abierta, lista_cerrada, nodos_camino)

In [83]:
def algoritmo_a_star(dibujar, grid, inicio, fin, filas, ancho_grid):
    contador = 0
    conjunto_abierto = PriorityQueue()
    conjunto_abierto.put((0, contador, inicio))
    came_from = {}
    
    lista_abierta = {inicio}
    lista_cerrada = set()
    
    inicio.g = 0
    inicio.h = h(inicio.get_pos(), fin.get_pos())
    inicio.f = inicio.h
    
    while not conjunto_abierto.empty():
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    pygame.quit()
                    return False
                
        actual = conjunto_abierto.get()[2]
        lista_abierta.remove(actual)
        lista_cerrada.add(actual)
        
        if actual != inicio and actual != fin:
            actual.marcar_visitado()
        
        if actual == fin:
            reconstruir_camino(came_from, fin, dibujar, lista_abierta, lista_cerrada)
            fin.marcar_camino()
            inicio.marcar_camino()
            
            nodos_camino = set()
            actual = fin
            while actual in came_from:
                nodos_camino.add(actual)
                actual = came_from[actual]
            nodos_camino.add(inicio)
            
            dibujar(lista_abierta, lista_cerrada, nodos_camino)
            
            mostrar_resultado = True
            while mostrar_resultado:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        return True
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_ESCAPE:
                            pygame.quit()
                            return True
                        mostrar_resultado = False
                        break
                
                dibujar(lista_abierta, lista_cerrada, nodos_camino)
                pygame.time.delay(100)
            
            return True
            
        for vecino, costo, direccion in actual.vecinos:
            if vecino in lista_cerrada:
                continue
                
            temp_g = actual.g + costo
            
            if temp_g < vecino.g:
                came_from[vecino] = actual
                vecino.g = temp_g
                vecino.h = h(vecino.get_pos(), fin.get_pos())
                vecino.f = vecino.g + vecino.h
                
                vecino.nodo_origen = actual
                
                if vecino not in lista_abierta:
                    contador += 1
                    conjunto_abierto.put((vecino.f, contador, vecino))
                    lista_abierta.add(vecino)
        
        dibujar(lista_abierta, lista_cerrada, set())
            
    return False


In [84]:
def crear_grid(filas, ancho):
    grid = []
    ancho_nodo = ancho // filas
    numero = 1
    
    for i in range(filas):
        grid.append([])
        for j in range(filas):
            nodo = Nodo(i, j, ancho_nodo, filas, numero)
            grid[i].append(nodo)
            numero += 1
            
    return grid

In [85]:
def dibujar_grid(ventana, filas, ancho):
    ancho_nodo = ancho // filas
    for i in range(filas):
        pygame.draw.line(ventana, GRIS, (0, i * ancho_nodo), (ancho, i * ancho_nodo), 2)  
        for j in range(filas):
            pygame.draw.line(ventana, GRIS, (j * ancho_nodo, 0), (j * ancho_nodo, ancho), 2) 

In [86]:
def dibujar_listas(ventana, lista_abierta, lista_cerrada, nodos_camino, ancho_grid, alto_ventana):
    area_listas = pygame.Rect(ancho_grid + 10, 0, ANCHO_VENTANA - ancho_grid - 10, alto_ventana)
    pygame.draw.rect(ventana, AZUL_CLARO, area_listas)
    pygame.draw.rect(ventana, NEGRO, area_listas, 2)

    titulo_abierta = FUENTE_LISTA.render("Lista Abierta:", True, NEGRO)
    ventana.blit(titulo_abierta, (ancho_grid + 20, 20))
    
    lista_abierta_ordenada = sorted(lista_abierta, key=lambda x: (x.f, -x.g))
    ids_abierta = [nodo.get_id() for nodo in lista_abierta_ordenada if nodo.f != float("inf")]
    
    x_pos = ancho_grid + 20
    y_pos = 50
    max_width = ANCHO_VENTANA - ancho_grid - 40 
    
    texto_inicio = FUENTE_LISTA.render("Lista Abierta: [", True, NEGRO)
    ventana.blit(texto_inicio, (x_pos, y_pos))
    x_pos += texto_inicio.get_width()
    
    for i, id_str in enumerate(ids_abierta):
        if i > 0: 
            coma = FUENTE_LISTA.render(", ", True, NEGRO)
            ventana.blit(coma, (x_pos, y_pos))
            x_pos += coma.get_width()
        
        id_texto = FUENTE_LISTA.render(id_str, True, ROJO if any(nodo.numero_secuencial == int(id_str) and nodo in nodos_camino for nodo in lista_abierta) else NEGRO)
        
        if x_pos + id_texto.get_width() > ancho_grid + max_width:
            y_pos += 25
            x_pos = ancho_grid + 40 
        
        ventana.blit(id_texto, (x_pos, y_pos))
        x_pos += id_texto.get_width()
    
    cierre = FUENTE_LISTA.render("]", True, NEGRO)
    ventana.blit(cierre, (x_pos, y_pos))

    y_pos += 50
    titulo_cerrada = FUENTE_LISTA.render("Lista Cerrada:", True, NEGRO)
    ventana.blit(titulo_cerrada, (ancho_grid + 20, y_pos))
    y_pos += 30
    
    x_pos = ancho_grid + 20
    texto_inicio = FUENTE_LISTA.render("Lista Cerrada: [", True, NEGRO)
    ventana.blit(texto_inicio, (x_pos, y_pos))
    x_pos += texto_inicio.get_width()
    
    lista_cerrada_ordenada = sorted(lista_cerrada, key=lambda x: x.numero_secuencial)
    
    for i, nodo in enumerate(lista_cerrada_ordenada):
        if nodo.f == float("inf"):
            continue
            
        if i > 0: 
            coma = FUENTE_LISTA.render(", ", True, NEGRO)
            ventana.blit(coma, (x_pos, y_pos))
            x_pos += coma.get_width()
        
        color = ROJO if nodo in nodos_camino else NEGRO
        
        id_texto = FUENTE_LISTA.render(nodo.get_id(), True, color)
        
        if x_pos + id_texto.get_width() > ancho_grid + max_width:
            y_pos += 25
            x_pos = ancho_grid + 40  
        
        ventana.blit(id_texto, (x_pos, y_pos))
        
        pygame.draw.line(ventana, color, 
                         (x_pos, y_pos + id_texto.get_height()//2), 
                         (x_pos + id_texto.get_width(), y_pos + id_texto.get_height()//2), 
                         1)
        
        x_pos += id_texto.get_width()
    
    cierre = FUENTE_LISTA.render("]", True, NEGRO)
    ventana.blit(cierre, (x_pos, y_pos))

In [87]:
def dibujar(ventana, grid, filas, ancho_grid, lista_abierta=None, lista_cerrada=None, nodos_camino=None):
    ventana.fill(BLANCO)
    for fila in grid:
        for nodo in fila:
            nodo.dibujar(ventana)

    dibujar_grid(ventana, filas, ancho_grid)
    
    if lista_abierta is not None and lista_cerrada is not None:
        if nodos_camino is None:
            nodos_camino = set()
        dibujar_listas(ventana, lista_abierta, lista_cerrada, nodos_camino, ancho_grid, ALTO_VENTANA)
    
    pygame.display.update()

In [88]:
def obtener_click_pos(pos, filas, ancho):
    ancho_nodo = ancho // filas
    x, y = pos
    col = x // ancho_nodo
    fila = y // ancho_nodo
    return fila, col


In [89]:
def main(ventana, ancho, alto):
    ancho_grid = alto 
    
    FILAS = 10
    grid = crear_grid(FILAS, ancho_grid)

    inicio = None
    fin = None

    corriendo = True
    iniciado = False

    while corriendo:
        dibujar(ventana, grid, FILAS, ancho_grid)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                corriendo = False
                
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    corriendo = False

            if iniciado:
                continue
                
            if pygame.mouse.get_pressed()[0]:  
                pos = pygame.mouse.get_pos()
                if pos[0] < ancho_grid:  
                    fila, col = obtener_click_pos(pos, FILAS, ancho_grid)
                    if fila < FILAS and col < FILAS:
                        nodo = grid[fila][col]
                        
                        if not inicio and nodo != fin:
                            inicio = nodo
                            inicio.hacer_inicio()

                        elif not fin and nodo != inicio:
                            fin = nodo
                            fin.hacer_fin()

                        elif nodo != fin and nodo != inicio:
                            nodo.hacer_muro()

            elif pygame.mouse.get_pressed()[2]:
                pos = pygame.mouse.get_pos()
                if pos[0] < ancho_grid:
                    fila, col = obtener_click_pos(pos, FILAS, ancho_grid)
                    if fila < FILAS and col < FILAS:
                        nodo = grid[fila][col]
                        nodo.restablecer()
                        if nodo == inicio:
                            inicio = None
                        elif nodo == fin:
                            fin = None
                    
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and not iniciado and inicio and fin:
                    for fila in grid:
                        for nodo in fila:
                            if not (nodo.es_muro or nodo.es_nodo_inicio or nodo.es_nodo_fin):
                                nodo.restablecer()
                    
                    for fila in grid:
                        for nodo in fila:
                            nodo.actualizar_vecinos(grid)
                    
                    iniciado = True
                    algoritmo_a_star(
                        lambda lista_a=None, lista_c=None, nodos_p=None: dibujar(ventana, grid, FILAS, ancho_grid, 
                                                            lista_a if lista_a is not None else set(), 
                                                            lista_c if lista_c is not None else set(),
                                                            nodos_p if nodos_p is not None else set()),
                        grid, inicio, fin, FILAS, ancho_grid
                    )
                    iniciado = False
                    
                if event.key == pygame.K_c:
                    inicio = None
                    fin = None
                    grid = crear_grid(FILAS, ancho_grid)

    pygame.quit()


In [90]:
if __name__ == "__main__":
    main(VENTANA, ANCHO_VENTANA, ALTO_VENTANA)

error: video system not initialized